In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *


In [55]:
exp = get_exp((39,269,329),directory_project)


begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [56]:
select_hyph = get_hyph_infos(exp)

In [12]:
len(exp.hyphaes)

3518

In [57]:
tracked = [hyph for hyph in exp.hyphaes if np.any(np.array([c[2] for c in select_hyph[hyph]])>=80)]

In [58]:
rh = [hyph for hyph in exp.hyphaes if np.any(np.array([c[2] for c in select_hyph[hyph]])>=150)]

In [59]:
rh2 = [hyph for hyph in exp.hyphaes if np.any(np.array([c[2] for c in select_hyph[hyph]])>=200)]

In [60]:
len(tracked),len(exp.hyphaes),len(rh),len(rh2)

(784, 3990, 351, 202)

In [84]:
hyph = choice(rh2)
speeds = [c[2] for c in select_hyph[hyph]]
ts = [c[0] for c in select_hyph[hyph]]
tp1s = [c[1] for c in select_hyph[hyph]]

In [85]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.plot(ts,speeds)
ax.set_xlabel('time (h)')
ax.set_ylabel('speed $\mu m .h^{-1}$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'speed $\\mu m .h^{-1}$')

In [83]:
plot_raw_plus(exp,hyph.ts[-1],[hyph.end.label]+[hyph.root.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
plot_raw_plus(exp,hyph.ts[0],[hyph.end.label]+[hyph.root.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
plot_raw_plus(exp,hyph.ts[0]+1,[hyph.end.label]+[hyph.root.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
hyph.end.ts()

[5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [14]:
displacement = []
for node in exp.nodes:
    for index,t in enumerate(node.ts()[:-1]):
        if node.degree(t)==3:
            tp1= node.ts()[index+1]
            displacement.append(np.linalg.norm(node.pos(t)-node.pos(tp1)))

In [18]:
np.mean(displacement)

5.03285403744039

In [17]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(displacement,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([3.9865e+04, 1.0610e+03, 1.9000e+01, 7.0000e+00, 6.0000e+00,
        9.0000e+00, 6.0000e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 2.0000e+00, 0.0000e+00,
        2.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+00]),
 array([  0.        ,  19.96140721,  39.92281442,  59.88422163,
         79.84562884,  99.80703605, 119.76844326, 139.72985047,
        159.69125767, 179.65266488, 199.61407209, 219.5754793 ,
        239.53688651, 259.49829372, 279.45970093, 299.42110814,
        319.38251535, 339.34392256, 359.30532977, 379.26673698,
        399.22814419, 419.1895514 , 439.15095861, 459.11236582,
        479.07377302, 499.03518023, 518.99658744, 538.95799465,
        558.91940186, 578.88080907, 598.84221628]),
 <a list of 30 Patch objects>)

In [21]:
deg3_nodes = [node for node in exp.nodes if node.degree(node.ts()[0])==3]
width_truple = []
for node in deg3_nodes:
    for t in node.ts():
        if node.degree(t)==3:
            edges = [(Edge(node,neighbour,exp),t) for neighbour in node.neighbours(t)]
            width_truple.append([edge.width(t) for edge,t in edges])

In [11]:
[5,8,7].sort()

In [22]:
dists = []
main = []
for truple in width_truple:
    truple.sort()
    dists.append(truple[1]-truple[0])
    dists.append(truple[2]-truple[1])
    main.append(truple[2])
    main.append(truple[2])

In [23]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.scatter(main,dists,s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
widths = []
for truple in width_truple:
    widths.append(truple[0])
    widths.append(truple[1])
    widths.append(truple[2])

In [27]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(widths,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([2.7300e+02, 2.0900e+02, 6.5800e+02, 1.2430e+03, 3.6250e+03,
        8.1050e+03, 1.0136e+04, 7.8260e+03, 4.5740e+03, 1.5740e+03,
        5.1600e+02, 1.9400e+02, 1.2400e+02, 9.0000e+01, 4.9000e+01,
        4.4000e+01, 5.0000e+01, 4.3000e+01, 2.1000e+01, 1.6000e+01,
        2.3000e+01, 2.9000e+01, 3.1000e+01, 1.4000e+01, 1.6000e+01,
        1.0000e+01, 1.8000e+01, 2.6000e+01, 1.3000e+01, 1.1000e+01]),
 array([ 0.        ,  1.14290332,  2.28580664,  3.42870997,  4.57161329,
         5.71451661,  6.85741993,  8.00032326,  9.14322658, 10.2861299 ,
        11.42903322, 12.57193655, 13.71483987, 14.85774319, 16.00064651,
        17.14354983, 18.28645316, 19.42935648, 20.5722598 , 21.71516312,
        22.85806645, 24.00096977, 25.14387309, 26.28677641, 27.42967974,
        28.57258306, 29.71548638, 30.8583897 , 32.00129302, 33.14419635,
        34.28709967]),
 <a list of 30 Patch objects>)

In [30]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

ax.hist(dists,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.4828e+04, 6.3580e+03, 2.6220e+03, 1.0880e+03, 5.1200e+02,
        3.2000e+02, 1.7000e+02, 1.0800e+02, 7.5000e+01, 4.6000e+01,
        3.0000e+01, 2.5000e+01, 2.4000e+01, 1.8000e+01, 2.4000e+01,
        2.4000e+01, 1.8000e+01, 1.7000e+01, 7.0000e+00, 1.1000e+01,
        1.0000e+01, 6.0000e+00, 1.6000e+01, 4.0000e+00, 7.0000e+00,
        2.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00]),
 array([ 0.        ,  0.99580817,  1.99161633,  2.9874245 ,  3.98323266,
         4.97904083,  5.97484899,  6.97065716,  7.96646532,  8.96227349,
         9.95808166, 10.95388982, 11.94969799, 12.94550615, 13.94131432,
        14.93712248, 15.93293065, 16.92873881, 17.92454698, 18.92035515,
        19.91616331, 20.91197148, 21.90777964, 22.90358781, 23.89939597,
        24.89520414, 25.8910123 , 26.88682047, 27.88262863, 28.8784368 ,
        29.87424497]),
 <a list of 30 Patch objects>)

In [41]:
ratios = []
main = []
for truple in width_truple:
    truple.sort()
    dist1= truple[1]-truple[0]
    dist2 = truple[2]-truple[1]
    if dist1>0.1:
        main.append(truple[2])
        ratios.append(dist2/dist1)

In [43]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.scatter(main,ratios,1)
# ax.hist(ratios,60,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
ratio_significant = [ratio for ratio in ratios if ratio<0.5]
ratio_significant2 = [ratio for ratio in ratios if ratio>2]

In [49]:
len(ratio_significant),len(ratios),len(ratio_significant2)

(5776, 12436, 2759)